### **Gọi thư viện**

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

#### Lấy thông tin của các bài hát trong một album

In [2]:
def init_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--start-maximized")  # Bật full màn hình
    driver = webdriver.Chrome(options=chrome_options)
    return driver
    

def close_popup(driver):
    try:
        close_button = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, "close-btn")))
        close_button.click()
        print("--> Popup đã đóng!")
        time.sleep(1)
    except:
        print("--> Không có popup!")

In [3]:
def get_song_info_from_portal(portal_element):
    # Tên bài hát
    try:
        song_title = portal_element.find_element(By.CLASS_NAME, "item-title").text
    except:
        song_title = None
    # Lượt thích
    try:
        likes = portal_element.find_element(By.XPATH, "//div[@class='stat-item']/i[contains(@class, 'ic-like')]/following-sibling::span").text
    except:
        likes = None
    # Lượt nghe
    try:
        views = portal_element.find_element(By.XPATH, "//div[@class='stat-item']/i[contains(@class, 'ic-view')]/following-sibling::span").text
    except:
        views = None
        
    return song_title, likes, views


def extract_info_from_submenu(submenu, label):
    try:
        return submenu.find_element(By.XPATH, f".//h3[contains(text(), '{label}')]/following-sibling::div").text
    except:
        return None
    
def get_submenu_info(driver, portal_element):
    actions = ActionChains(driver)
    try:
        song_image = portal_element.find_element(By.TAG_NAME, "img")  # Tìm hình ảnh xuất hiện trên portal
        actions.move_to_element(song_image).perform()  # Di chuyển chuột tới hình ảnh đó
        time.sleep(2)  

        submenu = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "submenu-content"))
        )

        artist = extract_info_from_submenu(submenu, "Nghệ sĩ")
        album = extract_info_from_submenu(submenu, "Album")
        composer = extract_info_from_submenu(submenu, "Sáng tác")
        genre = extract_info_from_submenu(submenu, "Thể loại")
        provided_by = extract_info_from_submenu(submenu, "Cung cấp bởi")

    except:
        print(f"Không thể hover vào hình ảnh bài hát.")
        artist, album, composer, genre, provided_by = None, None, None, None, None

    return artist, album, composer, genre, provided_by

In [4]:
def get_info_songs_album(driver, url, album_title):
    print(f"\n => Đang thu thập dữ liệu từ album: {album_title} | URL: {url}")

    # TRUY CẬP TRANG WEB CỦA ALBUM
    driver.get(url)
    time.sleep(5)
    
    # ĐÓNG POPUP (NẾU CÓ)
    close_popup(driver)
    
    # TÌM DANH SÁCH CÁC BÀI HÁT
    songs = driver.find_elements(By.CLASS_NAME, "media-left")[2:]
    song_data = []

    # LẤY THÔNG TIN CỦA TỪNG BÀI HÁT
    for index, song in enumerate(songs):  
        try:
            # 1. Click chuột phải vào tên Album của bài hát để mở menu ngữ cảnh (có chứa các thông tin về bài hát)
            actions = ActionChains(driver)
            actions.context_click(song).perform()
            time.sleep(2)

            # 2. Sau khi click chuột phải, tìm portal xuất hiện trên web
            portal_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "zm-portal"))
            )

            # 3. Lấy thông tin bài hát từ portal đó
            song_title, likes, views = get_song_info_from_portal(portal_element)

            # 4. Hover chuột vào hình ảnh để hiển thị submenu-content trên web
            artist, album, composer, genre, provided_by = get_submenu_info(driver, portal_element)

            # 5. Lưu vào danh sách
            song_data.append({
                "Album": album,
                "Tên bài hát": song_title,
                "Lượt yêu thích": likes,
                "Lượt nghe": views,
                "Nghệ sĩ": artist,
                "Sáng tác": composer,
                "Thể loại": genre,
                "Nhà cung cấp": provided_by
            })

            print(f"{index+1}. {song_title} | {artist} | {album} | {composer} | {genre} | {provided_by} | {likes} | {views}")

            # Đóng portal hiện tại để tránh lỗi khi click bài tiếp theo
            driver.execute_script("document.querySelector('.zm-portal').remove();")

        except Exception as e:
            print(f"Lỗi khi lấy bài hát {index}: {e}")

    return song_data

#### Duyệt qua tất cả bài hát có trong các album top 100 trên Zing mp3

In [5]:
def wait_album_list(driver):
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//h4[@class='title is-6']/a"))
        )
        print("Danh sách album đã xuất hiện!")
    except:
        print("Không tìm thấy danh sách album. Kiểm tra lại XPath hoặc chờ tải trang lâu hơn.")
        driver.quit()
        return False
    
    return True

def get_albums_at_present(driver, album_links, seen_albums):
    albums = driver.find_elements(By.XPATH, "//h4[@class='title is-6']/a")
    new_count = 0

    for album in albums:
        try:
            album_title = album.get_attribute("title").strip()
            album_url = album.get_attribute("href").strip()

            if album_url not in seen_albums:
                album_links.append((album_title, album_url))
                seen_albums.add(album_url)
                new_count += 1
        except:
            continue

    return new_count

def collect_albums(driver):
    album_links = []  # Dùng list để lưu theo thứ tự
    seen_albums = set()  # Dùng set để tránh trùng lặp

    while True:
        new_albums = get_albums_at_present(driver, album_links, seen_albums)
        if new_albums == 0:  # Nếu không có album mới, dừng lại
            break
        driver.execute_script("window.scrollBy(0, 600);")  # Cuộn xuống một chút
        time.sleep(1)

    print(f"Tổng số album tìm thấy: {len(album_links)}")
    
    return album_links

In [6]:
def crawling_songs():
    print("\nBắt đầu thu thập dữ liệu ...")
    driver = init_driver()

    # TRUY CẬP TRANG WEB CHỨA CÁC ALBUM TOP 100
    top100_url = "https://zingmp3.vn/top100"
    driver.get(top100_url)
    time.sleep(5)  # Chờ trang tải

    # ĐÓNG POPUP (NẾU CÓ)
    close_popup(driver)

    # 1. Chờ danh sách các album xuất hiện trên web
    if not wait_album_list(driver):
        return

    # 2. Lấy tất cả album có mặt trên web trước khi cuộn xuống lấy tiếp
    album_links = collect_albums(driver)

    if len(album_links) == 0: # Không có album
        print("Không tìm thấy album nào :(")
        return

    # 3. Lưu dữ liệu bài hát từ tất cả album
    all_song_data = []

    # 4. Duyệt qua từng album để lấy danh sách bài hát theo thứ tự
    for album_title, album_url in album_links:
        song_data = get_info_songs_album(driver, album_url, album_title)
        all_song_data.extend(song_data)

    # Đóng trình duyệt
    driver.quit()

    # Lưu tất cả bài hát
    df = pd.DataFrame(all_song_data)
    
    csv_filename = "songs_df.csv"
    df.to_csv(csv_filename, index=False, encoding="utf-8-sig")

    print(f"\n Dữ liệu từ tất cả album Top 100 đã được lưu vào tệp: {csv_filename}")

    return df




In [7]:
crawling_songs()


Bắt đầu thu thập dữ liệu ...
--> Popup đã đóng!
Danh sách album đã xuất hiện!
Tổng số album tìm thấy: 40

 => Đang thu thập dữ liệu từ album: Top 100 Bài Hát Nhạc Trẻ Hay Nhất | URL: https://zingmp3.vn/album/Top-100-Bai-Hat-Nhac-Tre-Hay-Nhat-Quang-Hung-MasterD-Tang-Duy-Tan-Tung-Duong-Duong-Domic/ZWZB969E.html
--> Không có popup!
1. Mất Kết Nối | Dương Domic | Dữ Liệu Quý (EP) | Dương Domic | Việt Nam, V-Pop | DAO Music Entertainment | 582K | 23.6M
2. Tái Sinh | Tùng Dương | MULTIVERSE | Tăng Duy Tân | Việt Nam, V-Pop | +84 Vietnam New Wave | 744K | 24.1M
3. Mộng Hoa Sim | Thiên Tú | Mộng Hoa Sim (Single) | Thiên Tú | Việt Nam, V-Pop | MIXUS | None | None
4. Anh Đau Từ Lúc Em Đi | Trần Mạnh Cường | Anh Đau Từ Lúc Em Đi (Single) | Trần Mạnh Cường | Việt Nam, V-Pop | Incom | 501K | 18.1M
5. Cảm Ơn Em | Đặng Thiên Chí | Cảm Ơn Em (Single) | Đặng Thiên Chí | Việt Nam, V-Pop | VIEENT Music | 279K | 10M
6. Ánh Mắt Biết Cười | Quang Hùng MasterD, Tăng Duy Tân | Ánh Mắt Biết Cười (Single) | Lê

,Album,Tên bài hát,Lượt yêu thích,Lượt nghe,Nghệ sĩ,Sáng tác,Thể loại,Nhà cung cấp
0,Dữ Liệu Quý (EP),Mất Kết Nối,582K,23.6M,Dương Domic,Dương Domic,"Việt Nam, V-Pop",DAO Music Entertainment
1,MULTIVERSE,Tái Sinh,744K,24.1M,Tùng Dương,Tăng Duy Tân,"Việt Nam, V-Pop",+84 Vietnam New Wave
2,Mộng Hoa Sim (Single),Mộng Hoa Sim,None,None,Thiên Tú,Thiên Tú,"Việt Nam, V-Pop",MIXUS
3,Anh Đau Từ Lúc Em Đi (Single),Anh Đau Từ Lúc Em Đi,501K,18.1M,Trần Mạnh Cường,Trần Mạnh Cường,"Việt Nam, V-Pop",Incom
4,Cảm Ơn Em (Single),Cảm Ơn Em,279K,10M,Đặng Thiên Chí,Đặng Thiên Chí,"Việt Nam, V-Pop",VIEENT Music
...,...,...,...,...,...,...,...,...
3591,None,Victory (Mike Batt Mix),None,None,Bond,None,"Hòa Tấu, Violin",Sony Music Entertainment
3592,None,Rolling In The Deep,None,None,The Piano Guys,None,"Hòa Tấu, Piano",Sony Music Entertainment
3593,Mộng Đẹp Ngày Xưa (Hòa Tấu Tranh Sáo Bầu Vol.1),Một Kiếp Phong Ba,None,None,Hòa Tấu,Nhạc Hoa,"Hòa Tấu, Nhạc Cụ Dân Tộc",RIAV
3594,Mộng Đẹp Ngày Xưa (Hòa Tấu Tranh Sáo Bầu Vol.1),Con Yêu,None,None,Hòa Tấu,None,"Hòa Tấu, Nhạc Cụ Dân Tộc",Hòa Tấu
